In [2]:
TMDB_API_KEY = #API KEY с https://developers.themoviedb.org/

In [3]:
import requests

def get_movie_data(tmdbId):  
    url = "https://api.themoviedb.org/3/movie/%s" % tmdbId
    params = {
        "api_key": TMDB_API_KEY,
        "external_source": "imdb_id"
    }
    resp = requests.get(url, params=params)
    return resp.json()

In [4]:
get_movie_data(555)

{'adult': False,
 'backdrop_path': None,
 'belongs_to_collection': None,
 'budget': 0,
 'genres': [{'id': 53, 'name': 'Thriller'}],
 'homepage': 'http://www.luecke-im-system.de/',
 'id': 555,
 'imdb_id': 'tt0442896',
 'original_language': 'en',
 'original_title': 'Absolut',
 'overview': 'Two guys against globalization want to plant a virus in the network of a finance corporation. On the day of the attack Alex has an accident and cannot remember anything.',
 'popularity': 0.026365,
 'poster_path': '/6YemisOilgHbBp6UtgoONHg8eJk.jpg',
 'production_companies': [{'id': 319, 'name': 'Frenetic Films'}],
 'production_countries': [{'iso_3166_1': 'CH', 'name': 'Switzerland'}],
 'release_date': '2005-04-20',
 'revenue': 0,
 'runtime': 94,
 'spoken_languages': [{'iso_639_1': 'fr', 'name': 'Français'}],
 'status': 'Released',
 'tagline': '',
 'title': 'Absolut',
 'video': False,
 'vote_average': 0.0,
 'vote_count': 0}

In [5]:
import pandas as pd
import numpy as np

In [6]:
links_df = pd.read_csv("/home/ubuntu/data/movielens/ml-latest-small/links.csv")
links_df.tail()

,movieId,imdbId,tmdbId
9120,162672,3859980,402672.0
9121,163056,4262980,315011.0
9122,163949,2531318,391698.0
9123,164977,27660,137608.0
9124,164979,3447228,410803.0


In [7]:
import time

movies = []
for cnt, movie_id in enumerate(links_df.movieId.unique()):
    tmdbId = int(links_df.loc[movie_id].tmdbId)
    data = get_movie_data(tmdbId)
    if len(data) == 0:
        continue
    movies.append({
        "movie_id": movie_id, 
        "overview": data["overview"],
        "title": data["title"]
    })
    # 40 requests every 10 second
    time.sleep(10 / 40)

ValueError: cannot convert float NaN to integer

In [ ]:
print(len(movies))
movies[:2]

In [ ]:
texts = [m["overview"] for m in movies]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf.fit(texts)

In [ ]:
target_m = movies[0]
target_m

In [ ]:
item_profile = tfidf.transform([target_m["overview"]])
item_profile

In [ ]:
ftr_id_to_term = {ftr_id: term for term, ftr_id in tfidf.vocabulary_.items()}

In [ ]:
for ftr_id, score in sorted(
    zip(item_profile.indices, item_profile.data),
    key=lambda x: x[1],
    reverse=True
):
    print(ftr_id_to_term[ftr_id], ":", score)

In [ ]:
r_df = pd.read_csv("/home/ubuntu/data/movielens/ml-latest-small/ratings.csv")
# посчитаем количество рейтингов на пользователя
r_per_user = r_df.userId.value_counts()
# оставим только 10 пользователей с минимум 50 оценками
good_uids = np.random.choice(r_per_user[r_per_user >= 50].index, size=10, replace=False)
good_uids

In [ ]:
uid = np.random.choice(good_uids)
uid

In [ ]:
# история рейтингов пользователя
uid_df = r_df[r_df.userId == uid]
uid_df.head()

In [ ]:
uid_df["liked"] = (uid_df.rating >= uid_df.rating.mean()).astype(int)
uid_df.head()

In [ ]:
uid_df.liked.value_counts()

In [ ]:
X = []
y = []

for t in uid_df.itertuples():
    for m in movies:
        if m["movie_id"] == t.movieId:
            print(m["title"], ":", t.liked)
            X.append(tfidf.transform([m["overview"]]))
            y.append(t.liked)
            break

In [ ]:
from scipy.sparse import vstack
X = vstack(X, 'csr')
y = np.array(y)

In [ ]:
sum(X.multiply(y)).data